# CHANGE THESE CONFIGS, THEN UPDATE MODEL INITIALIZATION

In [ ]:
MODEL_PATH = "./models/dqn/dqn_simple"
TENSORBOARD_LOG_DIR = "./models/dqn_simple/logs"
IMAGE_TAG = "dqn_simple_rewards"
IMAGE_DIR = "./images/dqn_simple"
RUNS_FILE = "./models/dqn_simple/dqn_simple_success_runs.pkl"

#### ENV CONFIGS ####
CONFIG = {
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 15,  # Number of other vehicles to observe
        "features": ["presence", "x", "y", "vx", "vy"],  # Observe position and velocity
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-10, 10],
            "vy": [-10, 10]
        },
        "absolute": False,
        "clip": False,
        "normalize": False
    },
    "action": {
        "type": "DiscreteMetaAction",  # Keep simple, 5 discrete actions
    },
    "simulation_frequency": 10,
    "policy_frequency": 10,
    "destination": "o1",
    "initial_vehicle_count": 20,
    "spawn_probability": 0.8,
    "ego_spacing": 25,
    "initial_lane_id": None,
    "controlled_vehicles": 1,
    "duration": 15,  # seconds
    "vehicles_density": 1.0,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.6],
    "scaling": 5.5 * 1.3,
    "normalize_reward": False
}

## Imports

In [4]:
import time
import torch
import pickle
import random
import gymnasium as gym

from fvcore.nn import FlopCountAnalysis

from stable_baselines3 import A2C, PPO, SAC, DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import ProgressBarCallback


from simple_intersection_env import SimpleIntersectionEnv
from custom_training_callback import RewardTrackingCallback

c:\Users\igor\Documents\UCSD\CSE190_Ammanbdrolu\cse-190-final-project\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## Register Env with Gymnasium

In [5]:
gym.envs.registration.register(
    id="simple-intersection-v0",
    entry_point="simple_intersection_env:SimpleIntersectionEnv",
)

## Create and Wrap Env

In [6]:
env = gym.make("simple-intersection-v0", render_mode='rgb_array', config=CONFIG)
env = DummyVecEnv([lambda: env])

## UPDATE HERE: Set Up Correct Model

In [7]:
# model = A2C(
#     "MlpPolicy",
#     env,
#     n_steps=5,
#     learning_rate=7e-4,
#     gamma=0.99,
#     gae_lambda=1.0,
#     ent_coef=0.01,
#     vf_coef=0.5,
#     max_grad_norm=0.5,
#     rms_prop_eps=1e-5,
#     use_rms_prop=True,
#     normalize_advantage=False,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device='cpu'
# )
print(torch.version.cuda)  # Should not be None
print(torch.backends.cudnn.enabled)  # Should be True if CUDA is usable
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print (device)
# chat
# model = DQN(
#     "MlpPolicy",
#     env,
#     learning_rate=1e-4,  # often smaller for DQN
#     buffer_size=15000,
#     learning_starts=1000,
#     batch_size=32,
#     tau=1.0,  # soft update coefficient
#     gamma=0.99,
#     train_freq=4,
#     target_update_interval=1000,
#     exploration_fraction=0.1,
#     exploration_final_eps=0.02,
#     max_grad_norm=10,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device=device
# )
# default
# model = DQN("MlpPolicy",
#     env,
#     learning_rate=1e-4,
#     gamma=0.99,
#     tensorboard_log=TENSORBOARD_LOG_DIR,
#     verbose=0,
#     device=device)
# docs example https://highway-env.farama.org/quickstart/
model = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=500,
              learning_starts=50,
              batch_size=32,
              gamma=0.8,
              train_freq=4,
              gradient_steps=1,
              target_update_interval=100,
              verbose=1,
              tensorboard_log=TENSORBOARD_LOG_DIR,
              device=device)

11.8
True
cuda
Using cuda device


## Training

In [8]:
reward_callback = RewardTrackingCallback(
    tag=IMAGE_TAG,
    path_dir=IMAGE_DIR
)

destinations = ["o1", "o2", "o3"]
steps = [5000 for _ in destinations]

for dest, steps in zip(destinations, steps):
    config = CONFIG.copy()
    config["destination"] = dest  # Change destination for each training phase
    env = gym.make("simple-intersection-v0", render_mode='rgb_array', config=CONFIG)
    env = DummyVecEnv([lambda: env])
    model.set_env(env)  # Update the model with the new environment
    model.learn(
        total_timesteps=steps,
        callback=[ProgressBarCallback(), reward_callback]
    )
    reward_callback.start_new_phase()
reward_callback.save_all_plot()

Logging to ./models/dqn/logs\DQN_21


Output()

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.255    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 77       |
|    time_elapsed     | 5        |
|    total_timesteps  | 392      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.089    |
|    n_updates        | 85       |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 85       |
|    time_elapsed     | 7        |
|    total_timesteps  | 645      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.142    |
|    n_updates        | 149      |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 12       |
|    fps              | 88       |
|    time_elapsed     | 9        |
|    total_timesteps  | 803      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.496    |
|    n_updates        | 188      |
----------------------------------


...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 48       |
|    fps              | 82       |
|    time_elapsed     | 44       |
|    total_timesteps  | 3676     |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.305    |
|    n_updates        | 3382     |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 52       |
|    fps              | 83       |
|    time_elapsed     | 46       |
|    total_timesteps  | 3892     |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.00252  |
|    n_updates        | 3436     |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 56       |
|    fps              | 83       |
|    time_elapsed     | 49       |
|    total_timesteps  | 4117     |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.00703  |
|    n_updates        | 3493     |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 60       |
|    fps              | 84       |
|    time_elapsed     | 53       |
|    total_timesteps  | 4516     |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.00226  |
|    n_updates        | 3592     |
----------------------------------


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 64       |
|    fps              | 84       |
|    time_elapsed     | 56       |
|    total_timesteps  | 4744     |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.0019   |
|    n_updates        | 3649     |
----------------------------------


### Save Model

In [9]:
model.save(MODEL_PATH)

## Evaluation

### Load Model

In [10]:
model = DQN.load(MODEL_PATH)

### Run Evaluation

In [11]:
collisions = 0
destination_arrivals = 0
success_count = 0
successful_flopcount = 0
episodes = 100

# Store successful runs for rendering
successful_runs = []

for eps in range(100):
    config = CONFIG.copy()
    config["destination"] = "o" + str(random.randint(1, 3))
    env = gym.make("simple-intersection-v0", render_mode='rgb_array', config=config)
    
    seed = random.randint(0, 10000)

    obs, _ = env.reset(seed=seed)
    episode_flops = 0
    done = False
    truncated = False
    episode_reward = 0
    trajectory = []

    while not (done or truncated):
        # Flop Counting
        input_tensor, _ = model.policy.obs_to_tensor(obs)
        flops = FlopCountAnalysis(model.policy, input_tensor)
        flops.unsupported_ops_warnings(False)
        flops = flops.total()
        episode_flops += flops

        action, _states = model.predict(obs, deterministic=True)
        trajectory.append((obs, action))  # Save for later render if successful
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward
    
    crashed = info.get("crashed", False)
    arrived = info.get("arrived", False)
    if crashed:
        collisions += 1
    if arrived:
        destination_arrivals += 1
    if (not crashed) and arrived:
        success_count += 1
        successful_flopcount += episode_flops
        successful_runs.append((seed, config.copy(), trajectory))

    print(f"Episode {eps + 1} finished, total reward: {episode_reward}, destination: {config['destination']}")
    time.sleep(1)

print(f"Total collisions: {collisions} out of {episodes} episodes")
print(f"Total destination arrivals: {destination_arrivals} out of {episodes} episodes")
if success_count > 0:
    print(f"FLOPS per successful episode: {successful_flopcount / success_count:.2}")
else:
    print("No successful episodes.")



The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 1 finished, total reward: -10.0, destination: o2


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 2 finished, total reward: -10.0, destination: o3


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

...

Episode 98 finished, total reward: 10.0, destination: o3


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 99 finished, total reward: -10.0, destination: o2


The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_target.q_net.1, q_net_target.q_net.2, q_net_target.q_net.3, q_net_target.q_net.4
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
q_net_target, q_net_target.features_extractor, q_net_target.features_extractor.flatten, q_net_target.q_net, q_net_target.q_net.0, q_net_tar

Episode 100 finished, total reward: -10.0, destination: o1
Total collisions: 65 out of 100 episodes
Total destination arrivals: 35 out of 100 episodes
FLOPS per successful episode: 9.3e+06


### Save Successful Runs

In [12]:
with open(RUNS_FILE, "wb") as f:
    pickle.dump(successful_runs, f)

### Load Successful Runs File

In [13]:
with open(RUNS_FILE, "rb") as f:
    successful_runs = pickle.load(f)

### Render Successful Episodes

In [15]:
for i, (seed, config, trajectory) in enumerate(successful_runs):
    print(f"\nRendering successful episode {i + 1}")
    env = gym.make("simple-intersection-v0", render_mode='human', config=config)
    obs, _ = env.reset(seed=seed)
    for obs, action in trajectory:
        env.step(action)
        env.render()
        time.sleep(0.05)
    env.close()


Rendering successful episode 1

Rendering successful episode 2

Rendering successful episode 3

Rendering successful episode 4

Rendering successful episode 5

Rendering successful episode 6

Rendering successful episode 7

Rendering successful episode 8

Rendering successful episode 9

Rendering successful episode 10

Rendering successful episode 11

Rendering successful episode 12

Rendering successful episode 13

Rendering successful episode 14

Rendering successful episode 15

Rendering successful episode 16

Rendering successful episode 17

Rendering successful episode 18

Rendering successful episode 19

Rendering successful episode 20

Rendering successful episode 21

Rendering successful episode 22

Rendering successful episode 23

Rendering successful episode 24

Rendering successful episode 25

Rendering successful episode 26

Rendering successful episode 27

Rendering successful episode 28

Rendering successful episode 29

Rendering successful episode 30

Rendering successf